In [1]:

import os 


In [2]:
%pwd

'd:\\ITI\\DS Track\\Deep Learning\\Projects\\Kidney Classification\\Kidney-Disease-Classification\\research'

In [3]:
#chdir --> change directory
os.chdir("../")
%pwd

'd:\\ITI\\DS Track\\Deep Learning\\Projects\\Kidney Classification\\Kidney-Disease-Classification'

# Constant file

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_url:str
    local_data_files:Path
    unzip_dir:Path

In [6]:
from src.CNNClassifierKidneyDiseases.constants import *


In [7]:
from src.CNNClassifierKidneyDiseases.utils.common import read_yaml , create_directories

In [8]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH , params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifact_root])


    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
        root_dir = config.root_dir,
        source_url = config.source_url,
        local_data_files = config.local_data_files,
        unzip_dir = config.unzip_dir
        )

        return data_ingestion_config


# Component File

In [9]:
import os
import urllib.request as request
import zipfile
from src.CNNClassifierKidneyDiseases import logger
from src.CNNClassifierKidneyDiseases.utils.common import get_size
import gdown

In [16]:
# the first thing that this class take is the config from the 
# DataClassConfiguration 
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config


    def download_file(self)->str:
        """
            Fetch Data from the url
        """

        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_files
            os.makedirs(self.config.root_dir , exist_ok=True)
            logger.info(f"Downloading Data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix ='https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id , zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e

    def extraxt_zip_file(self):

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_files , "r")as zip_ref:
            zip_ref.extractall(unzip_path)





# Pipeline

1. Download The file
2. Extract the zipfile


In [18]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    # it automatic take the args from the constant variales that assigned
    #those assignment : CONFIG_FILE_PATH , PARAMS_FILE_PATH
    data_ingestion = DataIngestion(config= data_ingestion_config)
    #data_ingestion.download_file()
    data_ingestion.extraxt_zip_file()
except Exception as e:
    raise e

[2025-07-05 21:20:06,411: INFO: common: yaml file config\config.yaml loaded successfully]
[2025-07-05 21:20:06,414: INFO: common: yaml file params.yaml loaded successfully]
[2025-07-05 21:20:06,418: INFO: common: Created Directory at : artifacts]
[2025-07-05 21:20:06,420: INFO: common: Created Directory at : artifacts/data_ingestion]
